# Installing Detectron 2

In [1]:
import torch, detectron2
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [2]:
%%capture

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random

# import some common libraries
import os

# import some common detectron2 utilities
from detectron2.data.datasets import register_coco_instances
from detectron2.utils import comm
from detectron2.data import MetadataCatalog
from detectron2.utils.logger import setup_logger
from detectron2.config import LazyConfig, instantiate
from detectron2.engine import DefaultTrainer, AMPTrainer, default_writers, hooks, default_setup, DefaultPredictor
from detectron2.engine.defaults import create_ddp_model
from detectron2.evaluation import inference_on_dataset, print_csv_format
from detectron2.engine import launch, default_argument_parser
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.model_zoo import get_config
from detectron2.utils.visualizer import Visualizer, ColorMode
import matplotlib.pyplot as plt

from PIL import Image


In [3]:
from detectron2.engine import DefaultTrainer
import datetime
import matplotlib.colors as colors

# Registering a Dataset

In [4]:
from detectron2.data.datasets import register_coco_instances


    
    
metadata = {
            'thing_classes': ['nonmitotic', 'mitotic'],
            'thing_dataset_id_to_contiguous_id': {0:0, 1:1}
                }

            
for d in ["train", "test"]:
    register_coco_instances(f"cellseg_{d}_1.9", metadata, 
                            f"/home/anishjv/cell_seg_classify/coco_conv_dataset_1.9/{d}/instances_cellseg_1.9_{d}.json", 
                            f"/home/anishjv/cell_seg_classify/coco_conv_dataset_1.9/{d}/images")


 # Setting Configurations
 

In [7]:
now = datetime.datetime.now()
cfg = LazyConfig.load("detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.py")
cfg.dataloader.train.dataset.names = 'cellseg_train_1.9'
cfg.dataloader.test.dataset.names = 'cellseg_test_1.9'

cfg.train.max_iter = 21750
cfg.dataloader.train.total_batch_size = 4

cfg.model.proposal_generator.pre_nms_topk = [18000, 12000]
cfg.model.proposal_generator.post_nms_topk = [3000, 2000]

cfg.model.roi_heads.batch_size_per_image = 512
cfg.model.roi_heads.num_classes = 2
#cfg.model.roi_heads.box_predictor.use_focal = True experimental 

cfg.model.pixel_mean = [85.52, 85.52, 85.52]
cfg.model.pixel_std = [11.66, 11.66, 11.6]

cfg.model.proposal_generator.anchor_generator.sizes = [
    [32],
    [64],
    [128],
    [256],
    [300]
]


cfg.train.output_dir = f'/scratch/ajitj_root/ajitj99/anishjv/logs/{now.strftime("%Y%m%d_%H%M%S")}_res50'
cfg.train.checkpointer.period = 2175
cfg.train.eval_period = 1087
cfg.dataloader.evaluator['max_dets_per_image'] = 2000





Augmentations


In [8]:
cfg.dataloader.train.mapper.augmentations.append(
    {
        '_target_': detectron2.data.transforms.Resize, 'shape': (512, 512)
    }
    
)


cfg.dataloader.train.mapper.augmentations.append(
    {
        '_target_': detectron2.data.transforms.Resize, 'shape': (512, 768)
    }
    
)


cfg.dataloader.train.mapper.augmentations.append(
    {
        '_target_': detectron2.data.transforms.RandomCrop,  'crop_type': 'relative', 'crop_size': (0.25, 0.5)
    }

)

cfg.dataloader.train.mapper.augmentations.append(
    {
        "_target_": detectron2.data.transforms.RandomContrast, 'intensity_max': 1.25, 'intensity_min': 0.75
    }
)


cfg.dataloader.train.mapper.augmentations.append(
    {
        '_target_': detectron2.data.transforms.RandomBrightness, 'intensity_max': 1.25, 'intensity_min': 0.75
    }
)


Saving the configs in yaml format


In [9]:
LazyConfig.save(cfg, 'paper_models/Res50-1.9/r50_1.9.yaml')